In [1]:
import merkletools
from web3 import Web3
from solcx import compile_files
import client.utils as utils

In [2]:
contract_id, compiled_sol = compile_files(["mismatch.sol"], output_values=['abi', 'bin']).popitem()

w3 = Web3(Web3.EthereumTesterProvider())

mismatch = w3.eth.contract(abi=compiled_sol['abi'], bytecode=compiled_sol['bin'])

tx_hash = mismatch.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

Mismatch = w3.eth.contract(address=tx_receipt.contractAddress, abi=compiled_sol['abi'])

In [3]:
mt1 = merkletools.MerkleTools(hash_type="keccak_256")
mt2 = merkletools.MerkleTools(hash_type="keccak_256")

mt1.add_leaf(['0', '1','1','2','3','5','8','13','21','34','55','89','144','233','377','610','987','1597','2584','4181','6765','10946','17711','28657','46368','75025','121393','196418','317811','514229','832040','1346269'], True)
mt2.add_leaf(['0', '1','1','2','3','5','8','13','21','34','89','144','233','377','610','987','1597','2584','4181','6765','10946','17711','28657','46368','75025','121393','196418','317811','514229','832040','1346269','2178309'], True)

mt1.make_tree()
mt2.make_tree()

In [4]:
tx_hash = Mismatch.functions.start_committed_bs(
    len(mt1.leaves), 
    '0x' + mt1.get_merkle_root(),
    '0x' + mt2.get_merkle_root(),
    w3.eth.accounts[1],
    w3.eth.accounts[2],
    '0x' + mt1.get_merkle_root()).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [5]:
import asyncio
mt = [mt1, mt2]

def handle_event(event, conractor):
    middle = event['args']['middle']
    Mismatch.functions.send_rc_proof(utils.translate_proof(mt[conractor-1], middle)).transact({'from': w3.eth.accounts[conractor]})
    print(f'conractor = {conractor} middle = {middle}')
    

async def log_loop_c1(event_filter, poll_interval):
    for x in range(10):
        for PairCreated in event_filter.get_new_entries():
            handle_event(PairCreated, 1)
        await asyncio.sleep(poll_interval)

async def log_loop_c2(event_filter, poll_interval):
    for x in range(10):
        for PairCreated in event_filter.get_new_entries():
            handle_event(PairCreated, 2)
        await asyncio.sleep(poll_interval)

async def main():
    event_filter_c1 = Mismatch.events.request_proof_of_middle.createFilter(fromBlock=1)
    event_filter_c2 = Mismatch.events.request_proof_of_middle.createFilter(fromBlock=1)

    await asyncio.gather(
        log_loop_c1(event_filter_c1, 2),
        log_loop_c2(event_filter_c2, 1.5))


await main()


conractor = 1 middle = 16
conractor = 2 middle = 16
conractor = 2 middle = 8
conractor = 1 middle = 8
conractor = 2 middle = 12
conractor = 1 middle = 12
conractor = 2 middle = 10
conractor = 1 middle = 10
conractor = 2 middle = 9
conractor = 1 middle = 9


In [140]:
event_filter = Mismatch.events.request_proof_of_committed_step.createFilter(fromBlock=1)
event_filter.get_new_entries()

[AttributeDict({'args': AttributeDict({'high': 10}),
  'event': 'request_proof_of_committed_step',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x2c13aa88a3c9cf358587f69c57ab4b56b3c343887a0d63b7c52ff7337557e5f6'),
  'address': '0xF2E246BB76DF876Cef8b38ae84130F4F55De395b',
  'blockHash': HexBytes('0xad32fa62f7861175f72e05eb5755aa15bac35843676fda440c0462a83bb6e8c3'),
  'blockNumber': 12})]